# TCP/IP Project - Part 1 (Group 07)

In this notebook we load our CSV file (chat messages between Noy and Elias),
simulate TCP/IP encapsulation (IP + TCP + payload) using a hexdump preview,
and generate traffic on loopback so we can capture it in Wireshark.


## Prerequisites
- **Linux/macOS**: `sudo jupyter lab` → capture `lo` in Wireshark → run `demo_send()`
- **Windows**:
  1) Install Wireshark/Npcap with *WinPcap API-compatible mode* + *Support loopback traffic*.
  2) `pip install scapy pandas`
  3) Run Jupyter **as Administrator**
  4) Capture **Npcap Loopback Adapter** in Wireshark
  5) Run `demo_send(iface='Npcap Loopback Adapter')`



## Step 1 — Load Your CSV (Input)
1. Place your CSV file (e.g., `group05_http_input.csv`) in the same folder as this notebook.
2. The CSV must contain the following columns: `msg_id, app_protocol, src_app, dst_app, message, timestamp`.
3. In the next cell, set `CSV_PATH` to your file name and run the cell.
4. Verify that the preview shows your rows correctly.


In [1]:

import pandas as pd
filename = "group07_chat_input.csv" # Replace 'path_to_your_file.csv' with the actual file path
messages_df = pd.read_csv(filename)
messages_df


,msg_id,app_protocol,src_app,dst_app,message,timestamp
0,1,CHAT,Noy,Server,HELLO NOY,0.01
1,2,CHAT,Server,Noy,INFO Welcome Noy!,0.02
2,3,CHAT,Noy,Server,CONNECT ELIAS,0.03
3,4,CHAT,Server,Elias,INFO Noy wants to chat with you,0.04
4,5,CHAT,Elias,Server,ACCEPT CHAT,0.05
5,6,CHAT,Noy,Elias,Hi Elias!,0.06
6,7,CHAT,Elias,Noy,Hi Noy!,0.07
7,8,CHAT,Noy,Elias,How are you?,0.08



## Step 2 — Validate the Schema
The notebook will automatically check the CSV header. If a required column is missing, you will see an error message.
- If validation fails, fix your CSV and re-run Step 1.
- If it passes, continue to Step 3.


In [2]:
def validate_csv_format(df: pd.DataFrame) -> None:
    required_columns = ["msg_id", "app_protocol", "src_app", "dst_app", "message", "timestamp"]
    for col in required_columns:
        if col not in df.columns:
            raise ValueError(f"Missing required column: {col}")

messages_df["message"] = messages_df["message"].fillna("")
validate_csv_format(messages_df)

print("CSV format is OK")


CSV format is OK



## Step 3 — Map to TCP/IP Layers (Encapsulation)
This notebook will map each application message to the TCP/IP stack:
- Application → Transport (e.g., TCP/UDP headers)
- Internet (IP headers)
- Link (frame headers/footers)

Just run the cell(s) in this section to see the derived structures.



In [3]:
# Imports we used for the packet build + (on Windows) sending with Scapy
import socket, struct, random, time, platform
from typing import Optional

IS_WINDOWS = (platform.system() == 'Windows')
try:
    from scapy.all import IP as SCAPY_IP, TCP as SCAPY_TCP, Raw as SCAPY_Raw, send as scapy_send, get_if_list
    HAVE_SCAPY = True
except Exception as e:
    HAVE_SCAPY = False
    SCAPY_IMPORT_ERR = e
IS_WINDOWS, HAVE_SCAPY

(True, True)

In [4]:
# checksum for headers (for the demo)
def checksum(data: bytes) -> int:
    if len(data) % 2:
        data += b'\0'
    res = sum(struct.unpack('!%dH' % (len(data)//2), data))
    while res >> 16:
        res = (res & 0xFFFF) + (res >> 16)
    return ~res & 0xFFFF

# simple hexdump so we can see bytes clearly
def hexdump(data: bytes, width: int=16):
    for i in range(0, len(data), width):
        chunk = data[i:i+width]
        hex_bytes = ' '.join(f'{b:02x}' for b in chunk)
        ascii_bytes = ''.join(chr(b) if 32 <= b < 127 else '.' for b in chunk)
        print(f"{i:04x}  {hex_bytes:<{width*3}}  {ascii_bytes}")


In [5]:
def build_ip_header(src_ip: str, dst_ip: str, payload_len: int, proto: int=socket.IPPROTO_TCP) -> bytes:
    version_ihl = (4 << 4) + 5
    tos = 0
    total_length = 20 + payload_len
    identification = random.randint(0, 65535)
    flags_fragment = 0
    ttl = 64
    header_checksum = 0
    src = socket.inet_aton(src_ip)
    dst = socket.inet_aton(dst_ip)
    ip_header = struct.pack('!BBHHHBBH4s4s',
                             version_ihl, tos, total_length, identification,
                             flags_fragment, ttl, proto, header_checksum,
                             src, dst)
    chksum = checksum(ip_header)
    ip_header = struct.pack('!BBHHHBBH4s4s',
                             version_ihl, tos, total_length, identification,
                             flags_fragment, ttl, proto, chksum,
                             src, dst)
    return ip_header


In [6]:
def build_tcp_header(src_ip: str, dst_ip: str, src_port: int, dst_port: int, payload: bytes=b'',
                     seq: Optional[int]=None, ack_seq: int=0, flags: int=0x02, window: int=65535) -> bytes:
    if seq is None:
        seq = random.randint(0, 0xFFFFFFFF)

    data_offset = (5 << 4)  # TCP header length (no options)
    checksum_tcp = 0
    urg_ptr = 0

    tcp_header = struct.pack('!HHLLBBHHH',
                             src_port, dst_port, seq, ack_seq,
                             data_offset, flags, window,
                             checksum_tcp, urg_ptr)

    pseudo_header = struct.pack('!4s4sBBH',
                                socket.inet_aton(src_ip), socket.inet_aton(dst_ip),
                                0, socket.IPPROTO_TCP,
                                len(tcp_header) + len(payload))

    chksum = checksum(pseudo_header + tcp_header + payload)

    tcp_header = struct.pack('!HHLLBBHHH',
                             src_port, dst_port, seq, ack_seq,
                             data_offset, flags, window,
                             chksum, urg_ptr)
    return tcp_header


### Cross‑Platform Transport
- Linux/macOS: raw sockets (we include the IP header)
- Windows: Scapy + Npcap fallback (raw TCP sockets are blocked by the OS)

In [7]:
class RawTcpTransport:
    def __init__(self, src_ip: str, dst_ip: str, src_port: int, dst_port: int, iface: Optional[str]=None):
        self.src_ip = src_ip
        self.dst_ip = dst_ip
        self.src_port = src_port
        self.dst_port = dst_port
        self.iface = iface
        self.windows_fallback = IS_WINDOWS
        if not self.windows_fallback:
            self.sock = socket.socket(socket.AF_INET, socket.SOCK_RAW, socket.IPPROTO_RAW)
        else:
            if not HAVE_SCAPY:
                raise RuntimeError(
                    f"Windows detected but Scapy is not available: {SCAPY_IMPORT_ERR}.\n"
                    "Install with: pip install scapy. Ensure Npcap is installed with loopback support."
                )

    def encapsulate(self, data: bytes, flags: int=0x02) -> bytes:
        tcp = build_tcp_header(self.src_ip, self.dst_ip, self.src_port, self.dst_port, data, flags=flags)
        ip  = build_ip_header(self.src_ip, self.dst_ip, len(tcp) + len(data))
        return ip + tcp + data

    def send(self, data: bytes, flags: int=0x02):
        if not self.windows_fallback:
            pkt = self.encapsulate(data, flags=flags)
            self.sock.sendto(pkt, (self.dst_ip, 0))
        else:
            scapy_pkt = SCAPY_IP(src=self.src_ip, dst=self.dst_ip)/SCAPY_TCP(sport=self.src_port, dport=self.dst_port, flags=flags)/SCAPY_Raw(data)
            chosen_iface = self.iface
            if chosen_iface is None and self.dst_ip in ("127.0.0.1", "::1"):
                chosen_iface = "Npcap Loopback Adapter"
            scapy_send(scapy_pkt, verbose=False, iface=chosen_iface)


In [8]:
# Preview packet structure
src_ip = '127.0.0.1'
dst_ip = '127.0.0.1'
src_port = random.randint(1024, 65535)
dst_port = 12345
payload = b'CHAT preview - Noy and Elias'
pkt_preview = build_ip_header(src_ip, dst_ip, 20 + len(payload)) + build_tcp_header(src_ip, dst_ip, src_port, dst_port, payload) + payload
hexdump(pkt_preview)


0000  45 00 00 44 a7 d5 00 00 40 06 d4 dc 7f 00 00 01   E..D....@.......
0010  7f 00 00 01 c1 da 30 39 4e 0a 54 b6 00 00 00 00   ......09N.T.....
0020  50 02 ff ff fc c0 00 00 43 48 41 54 20 70 72 65   P.......CHAT pre
0030  76 69 65 77 20 2d 20 4e 6f 79 20 61 6e 64 20 45   view - Noy and E
0040  6c 69 61 73                                       lias



## Step 4 — Capture in Wireshark
1. Start capture in Wireshark.
2. Run the transmit/simulation cells in this notebook.
3. Observe packets appearing in Wireshark (timing may vary by system).
4. Stop the capture and save the file as `.pcap`.

### Suggested Wireshark Filters
- `ip.addr == 127.0.0.1 && tcp.port == 12345`
- `tcp.flags.syn == 1 && tcp.flags.ack == 0 && tcp.port == 12345`
- `tcp.flags.push == 1 && tcp.flags.ack == 1 && tcp.port == 12345`



## Step 5 — Generate/Synthesize Traffic
The notebook will simulate the transmission of your messages. You do not need to change parameters unless instructed in comments.
- Make sure Wireshark is open and ready to capture on your active interface.
- Consider applying a simple filter (e.g., `tcp port 80` for HTTP) to focus the view.


In [9]:
# Settings for our capture (loopback + port 12345)
src_ip = '127.0.0.1'
dst_ip = '127.0.0.1'
src_port = random.randint(1024, 65535)
dst_port = 12345
iface = "\\Device\\NPF_Loopback"  # Set to "Npcap Loopback Adapter" on Windows if needed
transport = RawTcpTransport(src_ip, dst_ip, src_port, dst_port, iface=iface)

### Send Messages from CSV file 

Iterate over the rows and send message by message

In [10]:
for index, row in messages_df.iterrows():
    message = str(row["message"]).strip()
    if not message:
        continue

    transport.send(message.encode("utf-8"), flags=0x18)  # PSH+ACK
    time.sleep(0.1)

print("Done - check Wireshark (tcp.port == 12345)")


C:\Users\noy2n\AppData\Roaming\Python\Python310\site-packages\scapy\sendrecv.py:479: SyntaxWarning: 'iface' has no effect on L3 I/O send(). For multicast/link-local see https://scapy.readthedocs.io/en/latest/usage.html#multicast
  warnings.warn(


Done - check Wireshark (tcp.port == 12345)



## Step 6 — Analyze and Explain
In your **report**:
- Explain how the CSV application messages became packets/frames through encapsulation.
- Use **Wireshark screenshots** to illustrate headers, ports, and payloads.
- Link observations back to your CSV rows (e.g., message IDs).



## Deliverables Checklist
- [ ] CSV input file.
- [ ] Executed notebook (with outputs).
- [ ] Wireshark .pcap capture.
